# 1 Esperienza con Analog Discovery 2


Si riportano i nomi dei file dei dati raccolti in questa esperienza:
* misura01.txt: tensione costante;
* misura02.txt: tensione sinsoide;
* misura03.txt: Ch1Ch2 cortocircuitati;
* misura04.txt: Ch1 scollegato;
* misura05.txt: partitore rng5;
* misura06.txt: partitore rng4;
* misura07.txt: partitore rng3.1;

Per le parti seguenti sono necessari:
* Computer,
* Breadbord,
* AD2,
* Cavetti
* Resistenza 1kOhm nominale
* Resistenza 10kOhm nominale
* Tester


# Task4 
Misuriamo con multimetro e AD2 lo stesso segnale erogato da AD2.

In [2]:
import tdwf
import matplotlib.pyplot as plt
import numpy as np
import time

### Inizializzazioni
ad2 = tdwf.AD2() # dichiarazione dell'analog discovery 2
scope = tdwf.Scope(ad2.hdwf) # inizializzazione dell'oscilloscopio
wgen = tdwf.WaveGen(ad2.hdwf) # inizializzazione del generatore di funzioni

### Settings del generatore di funzioni
wgen.w1.offs = 2 # offset dell'onda generata
wgen.w1.func = tdwf.funcDC # tipo di segnale generato

### Inizio generazione segnale
wgen.w1.start()

### Setting dell'oscilloscopio
scope.fs = 1e6 # frequenza a cui plotta
scope.npt = 1000 # numero di punti memorizzati
scope.ch1.rng=5 # definisco il range (mi definisce anche la risoluzione minima su y = max/bit)

### Inizio sampling
time.sleep(0.5) # si aspetta mezzo secondo per assicurare di non misurare il transiente in cui il segnale non è costante
scope.sample() # inizia il sampling
data = np.column_stack((scope.time.vals, scope.ch1.vals)) 
np.savetxt("misura01.txt", data, delimiter="\t")
mean = np.mean(scope.ch1.vals)
error = np.sqrt(np.var(scope.ch1.vals))
### Plotting
plt.title("")
plt.plot(scope.time.vals, scope.ch1.vals)
plt.xlabel("Time [s]")
plt.ylabel("$\Delta V$[V]")
print(f"Risulta che: {mean} +- {error}")

Digilent WaveForms SDK versione 3.22.2


<>:34: SyntaxWarning: invalid escape sequence '\D'
<>:34: SyntaxWarning: invalid escape sequence '\D'
C:\Users\franc\AppData\Local\Temp\ipykernel_12744\3662023543.py:34: SyntaxWarning: invalid escape sequence '\D'
  plt.ylabel("$\Delta V$[V]")


Dispositivo #1 non presente...


C:\Users\franc\AppData\Local\Temp\ipykernel_12744\3662023543.py:34: SyntaxWarning: invalid escape sequence '\D'
  plt.ylabel("$\Delta V$[V]")


RuntimeError: Dispositivo non trovato

Otteniamo:
$$
    V_{DC, \text{mult}} = (2.000 \pm 0.002) \, \text{V},

    V_{DC, \text{AD2}} = (2.00 \pm 0.01) \, \text{V}
$$

E' da specificare che l'incertezza della misura con AD2 deriva dalla tolleranza dichiarata dal costruttore di $10 \text{mV}$ sulla misura: la varianza campione e l'incertezza di scala dichiarata dal costruttore sono entrambe dell'ordine del mV, dunque trascurabili.
Segue che le due misure sono compatibili.


# Task 5
E' stato costruito un partitore di tensione con rapporto  $\frac{V_{in}}{V_{out}}$ non troppo inferiore a $1$. il circuito è alimentato in continua da AD2.


La  misura è stata effettuata con un'acquisizione di AD2.

Il segnale in un uscita dal partitore, ci si aspetta che sia:
\begin{align}
    V_{\text{in}} = V_{\text{out}} (1+x)
\end{align}
Dove è stato posto $x = \frac{R_1}{R_2}$, che procederemo a stimare tramite fit non lineare ai minimi quadratici confrontandolo con il valore nominale. <br />



In [ ]:
import tdwf
import matplotlib.pyplot as plt
import numpy as np
import time

### Inizializzazioni
ad2 = tdwf.AD2() # dichiarazione dell'analog discovery 2
scope = tdwf.Scope(ad2.hdwf) # inizializzazione dell'oscilloscopio
wgen = tdwf.WaveGen(ad2.hdwf) # inizializzazione del generatore di funzioni

### Settings del generatore di funzioni
wgen.w1.offs = 1.5 # offset dell'onda generata
wgen.w1.func = tdwf.funcDC # tipo di segnale generato
wgen.w1.ampl = 0
### Inizio generazione segnale
wgen.w1.start()

### Setting dell'oscilloscopio
scope.fs = 1e6 # frequenza a cui plotta
scope.npt = 8192 # numero di punti memorizzati
scope.ch1.rng=5 # definisco il range (mi definisce anche la risoluzione minima su y = max/bit)
scope.ch2.rng=5 # definisco il range (mi definisce anche la risoluzione minima su y = max/bit)

### Inizio sampling
time.sleep(0.5) # si aspetta mezzo secondo per assicurare di non misurare il transiente in cui il segnale non è costante
scope.sample() # inizia il sampling
data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals)) 
np.savetxt("misura05.txt", data, delimiter="\t")
### Plotting
plt.title("")
plt.plot(scope.time.vals, scope.ch1.vals, label="Ch1")
plt.plot(scope.time.vals, scope.ch2.vals, label="Ch2")
plt.legend()
plt.xlabel("Time [s]")
plt.ylabel("$\Delta V$[V]")

Come trattato in *tests.jpynb*, il diverso valore di *rng* influenza la risoluzione minima dello strumento siccome un ADC che digitalizza un segnale analogico va a "mappare" l'intervallo in $2^{14}$ livelli, pertanto, a meno di limiti della scheda, la risoluzione dovrebbe essere:
\begin{align*}
    \frac{\text{rng}}{2^{14}}
\end{align*}

Tuttavia ,il manuale dichiara che la risoluzione minima che ha lo strumento è  $0.32$ mV nel caso in cui si stia usando
una scala  $\leq 0.5 \text{V}/\text{div}$ o $3.58$ mV nel caso in cui si stia usando una scala  $\geq 1 \text{V}/\text{div}$. 
Nel caso dei valori di rng usati si dovrebbe avere che:

<!DOCTYPE html>
<html lang="it">
<head>
  <meta charset="utf-8" />
  <title>Tabella</title>
  <style>
    table {
      border-collapse: collapse;
      width: 100%;
      max-width: 700px;
    }
    th, td {
      padding: 8px 12px;
      text-align: center;
      vertical-align: middle;
    }
    /* contenuto interno della cella grande: righe verticali */
    .big-cell {
      display: flex;
      flex-direction: column;
      justify-content: center; /* centra verticalmente */
      gap: 18px;               /* spazio tra le "sottorighe" */
      min-height: 120px;       /* regola l'altezza complessiva della cella */
    }
    .big-cell div {
      line-height: 1;
    }
    .empty { color: transparent; } /* per mantenere l'altezza senza mostrare testo */
  </style>
  <center>
    <table>
        <thead>
        <tr>
            <th>Valori ottenuti dai dati</th>
            <th>Valori attesi con la formula</th>
            <th>Valore manuale</th>
        </tr>
        </thead>
        <tbody>
        <tr>
            <td>0.34 mV</td>
            <td>0.3 mV</td>
            <!-- ultima colonna: un'unica cella che occupa tutte le righe dati -->
            <td rowspan="3">
            <div class="big-cell">
                <div class="empty">&nbsp;</div>        <!-- prima "sottoriga" vuota -->
                <div>0.32 mV</div>                    <!-- seconda "sottoriga" con valore -->
                <div class="empty">&nbsp;</div>        <!-- terza "sottoriga" vuota -->
            </div>
            </td>
        </tr>
        <tr>
            <td>0.34 mV</td>
            <td>0.2 mV</td>
        </tr>
        <tr>
            <td>0.34 mV</td>
            <td>0.2 mV</td>
        </tr>
        </tbody>
        </table>
    </center>
</html>
Il valore che ha senso utilizzare è quello che effettivamente emerge dai dati.
Inoltre risulta evidente che non serve ridurre il range e fare più fit in  quanto la risoluzione non può migliorare.

Infine abbiamo analizzato i dati acquisiti utilizzando Matlab: in primis abbiamo graficato la distribuzione iniziale del segnale misurato, rispettivamente, da *Ch1* e *Ch2* realizzando per ciascuno dei due un istogramma che riporta sulle ascisse il potenziale assunto e sulle ordinate il numero di conteggi di tale valore. Tali istogrammi sono salvati come *DistribuzioneIN.pdf* e *DistribuzioneOUT.pdf*. <br />
Come incertezza sui conteggi si è trovato ragionevole prendere la deviazione standard di una distribuzione Poissoniana: è ragionevole supporre che, entro certi limiti quale il fatto che si usa lo stesso strumento, ciascuna misura è un fenomeno indipente, di una quantità stazionaria e che i campionamenti non siano simultanei.<br />
Per entrambi i canali è stato fatto un best-fit con una gaussiana. Nel primo caso ${\Chi^2}_{norm} \sim 1$ e residui si comportano bene.
Nel secondo caso ${\Chi^2}_{norm} \sim 3$ e i residui sono correlati: ci sono regioni nei quali sono tutti sopra lo zero, e regioni in cui vale il contrario. Di conseguenza risulta esserci un errore sistematico.


Fare il best-fit con la tolleranza e l'errore di scala del costruttore (quest'ultimo è molto piccolo sia rispetto alla deviazione standard delle distribuzioni fittate sopra sia rispetto alla tolleranza;  dunque è trascurabile), porta a un'evidente sovrastima dell'incertezza: l'errore di $10 \text{mV}$ dichiarato è troppo grande. Supponiamo dunque che questo sia trascurabile, che il misurando sia costante (affermazione molto pericolosa in quanto è generato dallo stesso strumento che effettua la misura e che fluttua), supponiamo che l'errore sistematico sia trascurabile rispetto alle fluttuazioni gaussiane dei segnali misurati sopra, segue che è ragionevole come incertezza prendere la deviazione standard fittata per le gaussiane di sopra.

Per rispettare le ipotesi del fit ai minimi quadrati di Levensberg-Marquardt poniamo come ordinata $V_{Ch1}$ con la tecnica dell'errore efficace, infatti è vero che:
* E' rispettata la condizione che l'errore sulla x sia trascurabile.
* Perché il best-fit con la gaussiana sopra è ragionevole allora gli errori sono gaussiani.
* E' a posteriori verificabile l'indipendenza tra Ch1 e Ch2.
Infine, è stato graficato il modello di best-fit e le misure salvandolo come *fitPartitore.pdf*. <br />

Il parametro di best-fit $x^*$ è pari a:
$$
x^* = 0.10110 \pm 0.00002,
$$
mentre il valore stimato con i valori nominali risulta essere pari a:
$$
x = 0.10 \pm 0.03.
$$

Il $\Chi^2 \sim 1.5$ e i residui mostrano la presenza di un errore sistematico, segue che è molto probabile che non siano valide le ipotesi di indipendenza fatte sopra.
Concludiamo dunque che i valori risultano essere compatibili, entro i limiti sperimentali, con il valore atteso calcolato con i valori nominali.

Lo script Matlab per l'analisi dati è il seguente:

Setup e lettura dati 

cd(fileparts(matlab.desktop.editor.getActiveFilename));
tbl = readtable("misura05.txt", NumHeaderLines=3);
tbl.Properties.VariableNames = ["timestamp", "Ch1", "Ch2"];
t0 = tbl.timestamp(1);
tbl.t = seconds(tbl.timestamp - t0);
tbl.timestamp = [];

sCh1=0.010 * ones(size(tbl.Ch1))+0.0005*tbl.Ch1;
sCh2=0.010 * ones(size(tbl.Ch2))+0.0005*tbl.Ch1;


Plot dati temporali 

figure;
plot(tbl.t, [tbl.Ch1 tbl.Ch2] ,'.');
title("Misure di potenziale nei due canali");
legend("Ch1","Ch2");
xlabel("t [ms]");
ylabel("V_{Ch#} [V]");


Funzione gaussiana 

gauss = @(p,x) p(1)* exp(-((x-p(2))./(2*p(3))).^2);



Istogramma e fit Ch1 

pp1 = [1500, 1.5, 0.001];
dx1 = min(diff(unique(tbl.Ch1)));

figure;

% --- Primo subplot: istogramma + fit ---

h1 = histogram(tbl.Ch1, 'FaceColor', 'r','BinWidth', dx1);

cv1 = h1.BinCounts;
scv1 = sqrt(max(cv1,1));
binCenters1 = h1.BinEdges(1:end-1) + diff(h1.BinEdges)/2;
wcv1 = 1./max(cv1,1);

hold on;
[pph1,R1,J1,CovB1,MSE1] = nlinfit(binCenters1, cv1, gauss, pp1,'Weights',wcv1);

fprintf("pph1 = %.6f\n",pph1);
fprintf("s_pph1 = %.6f\n",sqrt(diag(CovB1)));
MSE1

xx1 = linspace(min(tbl.Ch1), max(tbl.Ch1), 1000);
plot(xx1, gauss(pph1, xx1), 'LineWidth', 3);
errorbar(binCenters1, cv1, scv1, "r", 'LineStyle','none');
hold off;

title("Potenziali misurati da Ch1");
xlabel("V_{Ch1} [V]");
ylabel("Conteggi");
legend("dati","fit", "Location","NE");

myfig = gcf; % get current figure
myfig.Position(3:4) = [600 600];
exportgraphics(gcf,"DistribuzioneIN.pdf",ContentType="vector");

% --- Secondo subplot: residui normalizzati ---

plot(binCenters1,R1 , '.r');
hold on;
plot(xx1, xx1*0, 'g-'); % linea zero
hold off;

title("Residui del Best-Fit");
xlabel("V_{Ch1} [V]");
ylabel("Residui normalizzati");
legend("residui","res=0", "Location","NE");
grid on;

myfig = gcf; % get current figure
myfig.Position(3:4) = [600 600];
exportgraphics(gcf,"DistribuzioneIN_residui.pdf",ContentType="vector");


Istogramma e fit Ch2
pp2 = [550, 1.3625, 0.0015];
dx2 = min(diff(unique(tbl.Ch2)));

figure;
h2 = histogram(tbl.Ch2, 'FaceColor', 'b','BinWidth', dx2);

cv2 = h2.BinCounts;
scv2 = sqrt(max(cv2,1));
binCenters2 = h2.BinEdges(1:end-1) + diff(h2.BinEdges)/2;
wcv2=1./max(cv2,1)

hold on;
[pph2,R2,J2,CovB2,MSE2] = nlinfit(binCenters2, cv2, gauss, pp2,'Weights',wcv2);

fprintf("pph2 = %.6f\n",pph2);
fprintf("s_pph2 = %.6f\n",sqrt(diag(CovB2)));
MSE2


xx2 = linspace(min(tbl.Ch2), max(tbl.Ch2), 1000);
plot(xx2, gauss(pph2, xx2), 'LineWidth', 3);
errorbar(binCenters2,cv2,scv2,"b")
hold off;


title("Potenziali misurati da Ch2");
xlabel("V_{Ch2} [V]");
ylabel("Conteggi");
legend("dati","fit", "Location","NE");

myfig = gcf; % get current figure
myfig.Position(3:4) = [600 600];
exportgraphics(gcf,"DistribuzioneOUT.pdf",ContentType="vector");


% --- Secondo subplot: residui normalizzati ---

plot(binCenters2,R2 , '.r');
hold on;
plot(xx2, xx2*0, 'g-'); % linea zero
hold off;

title("Residui del Best-Fit");
xlabel("V_{Ch2} [V]");
ylabel("Residui normalizzati");
legend("residui","res=0", "Location","NE");
grid on;

myfig = gcf; % get current figure
myfig.Position(3:4) = [600 600];
exportgraphics(gcf,"DistribuzioneOUT_residui.pdf",ContentType="vector");

 Fit pesato Ch1 vs Ch2 con nlinfit 

sCh1eff=sqrt(pph1(3)^2+pph2(3)^2)*ones(length(tbl.Ch1),1);
sCh2=pph2(3)*ones(length(tbl.Ch1),1);

%% --- Varianze campionarie punto per punto ---

weights = 1 ./ (sCh1eff.^2);  % vettore di pesi

modelfun = @(b,x) (x .* (1 + b(1)));
beta0 = [0.1];

[mdl,R,J,CovB,MSE] = nlinfit(tbl.Ch2, tbl.Ch1, modelfun, beta0, 'Weights', weights);
mdl
s_mdl=sqrt(CovB)
MSE

%% --- Plot fit pesato con errorbar ---
figure;
errorbar(tbl.Ch2, tbl.Ch1,sCh1eff,sCh1eff,sCh2,sCh2, '.', 'DisplayName', 'dati'); 
hold on;
xx = linspace(min(tbl.Ch2), max(tbl.Ch2), 1000);
yy = modelfun(mdl, xx);
plot(xx, yy', 'r-', 'LineWidth', 2, 'DisplayName', 'Fit');
hold off;

legend("Location","NE");
title("V_{Ch2} vs V_{Ch1} (Fit)");
xlabel("V_{Ch2} [V]");
ylabel("V_{Ch1} [V]");

%% --- Calcolo chi^2 ridotto ---

myfig = gcf; % get current figure
myfig.Position(3:4) = [600 600];
exportgraphics(gcf,"FitPartitore.pdf",ContentType="vector");

xxx=linspace(min(tbl.Ch2),max(tbl.Ch2),1000)
plot(tbl.Ch2,R , '.r');
hold on;
plot(xx2, xx2*0, 'g-'); % linea zero
hold off;


title("Residui del Best-Fit");
xlabel("V_{Ch2} [V]");
ylabel("V_{Ch1} [V]");
legend("residui","res=0", "Location","NE");
grid on;
exportgraphics(gcf,"FitPartitore_residui.pdf",ContentType="vector");


